In [2]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Activation, Flatten, LeakyReLU, Dropout
from keras.utils import plot_model
from nnSetup import NUM_PLANETS, NUM_FEATURES, NUM_OUTPUTS

from kegra.layers.graph import GraphConvolution
from kegra.utils import *

FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization

RELU_ALPHA = 0.05

# INPUTS
inputs = Input(shape=(NUM_PLANETS, NUM_FEATURES))

# BODY
# TODO: insert residual GCNNs layers here
body = Dropout(0.2)(inputs)
body = Dense(10)(body)
body = LeakyReLU(alpha=RELU_ALPHA)(body)

body = Dropout(0.2)(body)
body = Dense(8)(body)
body = LeakyReLU(alpha=RELU_ALPHA)(body)




body = Flatten()(body)

# POLICY HEAD
policyHead = Dropout(0.2)(body)
policyHead = Dense(100)(policyHead)
policyHead = LeakyReLU(alpha=RELU_ALPHA)(policyHead)

policyHead = Dropout(0.2)(body)
policyHead = Dense(100)(policyHead)
policyHead = LeakyReLU(alpha=RELU_ALPHA)(policyHead)

policyHead = Dropout(0.2)(body)
policyHead = Dense(100)(policyHead)
policyHead = LeakyReLU(alpha=RELU_ALPHA)(policyHead)

policyHead = Dense(NUM_OUTPUTS, activation='softmax', name='policy')(policyHead)

# VALUE HEAD
valueHead = Dropout(0.3)(body)
valueHead = Dense(20)(valueHead)
valueHead = LeakyReLU(alpha=RELU_ALPHA)(valueHead)

valueHead = Dropout(0.3)(valueHead)
valueHead = Dense(15)(valueHead)
valueHead = LeakyReLU(alpha=RELU_ALPHA)(valueHead)

valueHead = Dropout(0.2)(valueHead)
valueHead = Dense(7)(valueHead)
valueHead = LeakyReLU(alpha=RELU_ALPHA)(valueHead)

valueHead = Dropout(0.2)(valueHead)
valueHead = Dense(1, activation='sigmoid', name='value')(valueHead)

# TODO: multiple percentages
model = Model(inputs=inputs, outputs=[policyHead, valueHead], name='GZ_DenseNetwork')


print(model.summary())
plot_model(model, to_file='model_graph.png')

# TODO: are these losses right??
# TODO: measure losses to get better loss weights?
# TODO: are these metrics right?
model.compile(loss=['categorical_crossentropy', 'binary_crossentropy'], loss_weights=[1, 2], optimizer='adam', metrics = ["accuracy"])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 22, 14)       0                                            
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 22, 14)       0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 22, 10)       150         dropout_10[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_9 (LeakyReLU)       (None, 22, 10)       0           dense_9[0][0]                    
__________________________________________________________________________________________________
dropout_11

In [4]:
from trainingGame import loadTrainingGame
from trainingHelper import TrainingHelper

game = loadTrainingGame("testGame.pickle")
helper = TrainingHelper(game) 

trainX = helper.getTrainX()



In [5]:
import numpy as np

model.predict(np.array([trainX[0]]))

[array([[0.00107294, 0.00093284, 0.00107461, 0.00113132, 0.00107199,
         0.00107634, 0.00128456, 0.00097383, 0.00121425, 0.0012838 ,
         0.00137838, 0.00125878, 0.00115916, 0.00088388, 0.00107319,
         0.00091255, 0.00113062, 0.00101098, 0.00128123, 0.00096877,
         0.00108837, 0.00119957, 0.00112611, 0.00118664, 0.00091671,
         0.00092098, 0.00118476, 0.0011504 , 0.0009681 , 0.00108217,
         0.00094565, 0.00118002, 0.00107051, 0.00112769, 0.00108426,
         0.00125077, 0.00123236, 0.00101   , 0.00110811, 0.00117735,
         0.00092882, 0.00105772, 0.00118676, 0.00108165, 0.00110044,
         0.00118064, 0.00110189, 0.00100813, 0.00134755, 0.00098648,
         0.00093789, 0.0011036 , 0.00097268, 0.00094883, 0.00113756,
         0.00105065, 0.00121832, 0.00109778, 0.00115205, 0.001259  ,
         0.00115686, 0.00101418, 0.00094448, 0.00102198, 0.0011504 ,
         0.00123028, 0.00107131, 0.00099855, 0.00100464, 0.00104993,
         0.00128492, 0.00094269, 0

In [6]:
model.save("gz_dev.model.new")